In [1]:
from hero_db_utils.datamodels import datamodel
import dotenv
dotenv.load_dotenv(override=True);

In [2]:
from hero_db_utils.datamodels.fields import AutoSerialField

@datamodel
class Triangle:
    sidea:float
    sideb:float
    sidec:int
    id:AutoSerialField() = None

    @property
    def is_equilateral(self):
        return self.sidea == self.sideb == self.sidec
    
    @property
    def sides(self):
        return [self.sidea, self.sideb, self.sidec]


In [3]:
mytriangles = Triangle.collection([Triangle(5, 5, "5"), Triangle(10,5,2), Triangle(2,2,2)])
mytriangles.asdf()

,sidea,sideb,sidec,id,group_id
0,5.0,5.0,5.0,NaN,NaN
1,10.0,5.0,2.0,NaN,NaN
2,2.0,2.0,2.0,NaN,NaN


In [26]:
mytriangles.save()

In [5]:
Triangle.objects().all()

,id,sidea,sideb,sidec
0,1,5.0,5.0,5.0
1,2,10.0,5.0,2.0
2,3,2.0,2.0,2.0


In [6]:
Triangle(1,1,2).save()

Triangle(sidea=1.0, sideb=1.0, sidec=2, id=None)

In [7]:
Triangle.objects().all()

,id,sidea,sideb,sidec
0,1,5.0,5.0,5.0
1,2,10.0,5.0,2.0
2,3,2.0,2.0,2.0
3,4,1.0,1.0,2.0


In [8]:
model, created = Triangle.objects().get_or_create(sidea=1, sideb=1, sidec=2)
model.data

sidea    1.0
sideb    1.0
sidec    2.0
id       NaN
dtype: float64

In [9]:
model, created = Triangle.objects().get_or_create(sidea=12, sideb=13, sidec=22)
created

True

In [10]:
Triangle.objects().all()

,id,sidea,sideb,sidec
0,1,5.0,5.0,5.0
1,2,10.0,5.0,2.0
2,3,2.0,2.0,2.0
3,4,1.0,1.0,2.0
4,5,1.0,1.0,2.0
5,6,12.0,13.0,22.0


Fetch related example:

In [11]:
from hero_db_utils.datamodels import datamodel
from hero_db_utils.datamodels.fields import ForeignKeyField, AutoSerialField
import pandas as pd
import dotenv
dotenv.load_dotenv(override=True);

@datamodel
class Store:
    manager_staff_id:ForeignKeyField("staff", "staff_id")
    address_id:ForeignKeyField("address","address_id")
    last_update:pd.to_datetime
    store_id:AutoSerialField() = None

@datamodel
class Address:
    address:str
    address2:str
    district:str
    city_id:ForeignKeyField("city", "city_id")
    postal_code:str
    phone:str
    last_update:pd.to_datetime
    address_id:AutoSerialField() = None

@datamodel
class City:
    city:str
    country_id:ForeignKeyField("country", "country_id")
    last_update:str
    city_id:AutoSerialField() = None

@datamodel
class Country:
    country:str
    last_update:pd.to_datetime
    country_id:AutoSerialField() = None


True

In [12]:
%env HERO_POSTGRES_DBNAME=dvd_rentals

env: HERO_POSTGRES_DBNAME=dvd_rentals


In [19]:
Store.objects().all()

,store_id,manager_staff_id,address_id,last_update
0,1,1,1,2006-02-15 09:57:12
1,2,2,2,2006-02-15 09:57:12


Join on related table for column "address_id":

In [18]:
Store.objects().fetch_related("address_id")

,store_id,manager_staff_id,address_id,last_update,address_id,address,address2,district,city_id,postal_code,phone,last_update
0,1,1,1,2006-02-15 09:57:12,1,47 MySakila Drive,None,Alberta,300,,,2006-02-15 09:45:30
1,2,2,2,2006-02-15 09:57:12,2,28 MySQL Boulevard,None,QLD,576,,,2006-02-15 09:45:30
